# Inference

This script showcases the different models available in TRIDENT and how to use them efficiently.

In [1]:
import pandas as pd
import torch
import pandas as pd
import numpy as np
import sys
from tqdm.notebook import tqdm
sys.path.insert(1, '/cephyr/users/skall/Alvis/Ecotoxformer/inference_scripts_for_website_2/TRIDENT/tutorials/inference_utils/')
sys.path.insert(1, '/cephyr/users/skall/Alvis/Ecotoxformer/inference_scripts_for_website_2/TRIDENT/development/development_utils/')
from inference_utils.TRIDENT_for_inference import TRIDENT_for_inference

Specify the model version and load the model

In [2]:
pd.__version__

'1.1.4'

# For ZENODO

In [2]:
root = '/cephyr/users/skall/Alvis/Ecotoxformer/inference_scripts_for_website_2/TRIDENT/'

In [3]:

for MODEL_ENDPOINT in ['EC50', 'EC10', 'EC50EC10']:
    for SPECIES in tqdm(['fish','invertebrates','algae']):

        MODEL_VERSION = f'{MODEL_ENDPOINT}_{SPECIES}'
        print(f'Running predictions on: {MODEL_ENDPOINT}_{SPECIES}\n')
        print('\t Loading model...\n')
        trident = TRIDENT_for_inference(model_version=MODEL_VERSION, path_to_model_weights=root+'TRIDENT/')
        trident.load_fine_tuned_model()
        print('\t Loading data...\n')
        data = pd.read_excel(root+'data/Preprocessed_complete_data.xlsx', sheet_name=f'{MODEL_ENDPOINT}_{SPECIES}')
        data['species_group'] = data.species_group.replace('crustaceans', 'invertebrates')
        
        print('\t Predicting...\n')
        results = trident.predict_toxicity(SMILES = data['SMILES'].tolist(), exposure_duration=data['Duration_Value'].tolist(), endpoint=data['endpoint'].tolist(), effect=data['effect'], return_cls_embeddings=False)

        results['exposure_duration'] = 10**results.exposure_duration

        data = data.sort_values(by=['SMILES','Duration_Value','effect','endpoint'])
        results = results.sort_values(by=['SMILES','exposure_duration','effect','endpoint'])

        data = pd.concat([data,results[['predictions log10(mg/L)','predictions (mg/L)']]], axis=1)

        data = data.sort_index()
        print(f'Saving...\n')
        data.to_csv(root+f'data/Preprocessed_complete_data_pred_{MODEL_ENDPOINT}_{SPECIES}.csv')

  0%|          | 0/3 [00:00<?, ?it/s]

Running predictions on: EC50_fish

	 Loading model...

	 Loading data...

	 Predicting...

Did not return onehotencoding for Endpoint. Why? You specified only one Endpoint or you specified NOEC and EC10 which are coded to be the same endpoint.
Did not return onehotencoding for Effect. Why? You specified only one Effect.
Will use input 0 to network due to no Onehotencodings being present.



100%|██████████| 206/206 [01:14<00:00,  2.76it/s]


Saving...

Running predictions on: EC50_invertebrates

	 Loading model...

	 Loading data...

	 Predicting...

Did not return onehotencoding for Endpoint. Why? You specified only one Endpoint or you specified NOEC and EC10 which are coded to be the same endpoint.



100%|██████████| 137/137 [00:51<00:00,  2.66it/s]


...

Saving...

Running predictions on: EC50EC10_invertebrates

	 Loading model...

	 Loading data...

	 Predicting...

Renamed NOEC *EC10* in 0 positions



100%|██████████| 197/197 [01:16<00:00,  2.59it/s]


Saving...

Running predictions on: EC50EC10_algae

	 Loading model...

	 Loading data...

	 Predicting...

Renamed NOEC *EC10* in 0 positions
Did not return onehotencoding for Effect. Why? You specified only one Effect.



100%|██████████| 98/98 [00:44<00:00,  2.21it/s]


Saving...



# For website

In [4]:
species_groups = ['fish', 'algae', 'invertebrates']
models = ['EC50EC10', 'EC50', 'EC10']
effectordering = {
            'EC50_algae': {'POP':'POP'},
            'EC10_algae': {'POP':'POP'},
            'EC50EC10_algae': {'POP':'POP'}, 
            'EC50_invertebrates': {'MOR':'MOR','ITX':'ITX'},
            'EC10_invertebrates': {'MOR':'MOR','DVP':'DVP','ITX':'ITX', 'REP': 'REP', 'MPH': 'MPH', 'POP': 'POP'} ,
            'EC50EC10_invertebrates': {'MOR':'MOR','DVP':'DVP','ITX':'ITX', 'REP': 'REP', 'MPH': 'MPH', 'POP': 'POP'} ,
            'EC50_fish': {'MOR':'MOR'},
            'EC10_fish': {'MOR':'MOR','DVP':'DVP','ITX':'ITX', 'REP': 'REP', 'MPH': 'MPH', 'POP': 'POP','GRO': 'GRO'} ,
            'EC50EC10_fish': {'MOR':'MOR','DVP':'DVP','ITX':'ITX', 'REP': 'REP', 'MPH': 'MPH', 'POP': 'POP','GRO': 'GRO'} 
            }

endpointordering = {
            'EC50_algae': {'EC50':'EC50'},
            'EC10_algae': {'EC10':'EC10'},
            'EC50EC10_algae': {'EC50':'EC50', 'EC10': 'EC10'}, 
            'EC50_invertebrates': {'EC50':'EC50'},
            'EC10_invertebrates': {'EC10':'EC10'},
            'EC50EC10_invertebrates': {'EC50':'EC50', 'EC10': 'EC10'},
            'EC50_fish': {'EC50':'EC50'},
            'EC10_fish': {'EC10':'EC10'},
            'EC50EC10_fish': {'EC50':'EC50', 'EC10': 'EC10'} 
            }

default_durations = {
    'algae': 72,
    'fish': 96,
    'invertebrates': 48
}

In [5]:
raw_data = pd.read_excel(root+'data/Preprocessed_complete_data.xlsx', sheet_name='dataset')

raw_data['species_group'] = raw_data.species_group.replace('crustaceans', 'invertebrates')

In [6]:
SMILES_COLUMN_NAME = 'SMILES_Canonical_RDKit'
for SPECIES in tqdm(species_groups):
    for model in models:
        cls_dict = {}
        MODEL_VERSION = f'{model}_{SPECIES}'
        EXPOSURE_DURATION = default_durations[SPECIES]
        PREDICTION_ENDPOINT = list(endpointordering[MODEL_VERSION].keys())[0]
        PREDICTION_EFFECT = list(effectordering[MODEL_VERSION].keys())[0]
        trident = TRIDENT_for_inference(model_version=MODEL_VERSION, path_to_model_weights=root+'TRIDENT/')
        trident.load_fine_tuned_model()
        data = raw_data.copy()
        
        data = data.drop_duplicates(subset=['SMILES_Canonical_RDKit']).dropna(subset=['SMILES_Canonical_RDKit'])

        results = trident.predict_toxicity(SMILES = data[SMILES_COLUMN_NAME].tolist(), exposure_duration=EXPOSURE_DURATION, endpoint=[PREDICTION_ENDPOINT]*len(data), effect=[PREDICTION_EFFECT]*len(data), return_cls_embeddings=True)
        results.reset_index(drop=True, inplace=True)
        results['CLS_embeddings'] = results['CLS_embeddings'].apply(lambda x: np.asarray(x, dtype=np.float32))
        results = results[['SMILES_Canonical_RDKit','CLS_embeddings']]        
        results.to_pickle(root+f'data/{MODEL_VERSION}_CLS_embeddings.pkl.zip', compression='zip')



  0%|          | 0/3 [00:00<?, ?it/s]

Renamed NOEC *EC10* in 0 positions



100%|██████████| 26/26 [00:16<00:00,  1.58it/s]


Did not return onehotencoding for Endpoint. Why? You specified only one Endpoint or you specified NOEC and EC10 which are coded to be the same endpoint.
Did not return onehotencoding for Effect. Why? You specified only one Effect.
Will use input 0 to network due to no Onehotencodings being present.



100%|██████████| 26/26 [00:16<00:00,  1.57it/s]


...


100%|██████████| 26/26 [00:16<00:00,  1.55it/s]


Did not return onehotencoding for Endpoint. Why? You specified only one Endpoint or you specified NOEC and EC10 which are coded to be the same endpoint.



100%|██████████| 26/26 [00:16<00:00,  1.54it/s]


Renamed NOEC *EC10* in 0 positions
Did not return onehotencoding for Endpoint. Why? You specified only one Endpoint or you specified NOEC and EC10 which are coded to be the same endpoint.



100%|██████████| 26/26 [00:16<00:00,  1.55it/s]


In [7]:
combined_results = raw_data.drop_duplicates(subset=['SMILES_Canonical_RDKit']).dropna(subset=['SMILES_Canonical_RDKit'])[[SMILES_COLUMN_NAME]]

In [ ]:
SMILES_COLUMN_NAME = 'SMILES_Canonical_RDKit'

combined_results = raw_data.drop_duplicates(subset=['SMILES_Canonical_RDKit']).dropna(subset=['SMILES_Canonical_RDKit'])[[SMILES_COLUMN_NAME]]

for SPECIES in tqdm(species_groups):
    
    for model in models:
        MODEL_VERSION = f'{model}_{SPECIES}'
        trident = TRIDENT_for_inference(model_version=MODEL_VERSION, path_to_model_weights=root+'TRIDENT/', device='cuda')
        trident.load_fine_tuned_model()
        data = raw_data.copy()
        
        for PREDICTION_ENDPOINT in endpointordering[MODEL_VERSION]:
            for PREDICTION_EFFECT in effectordering[MODEL_VERSION]:
                print(MODEL_VERSION, PREDICTION_ENDPOINT, PREDICTION_EFFECT)
                try:
                    EXPOSURE_DURATION = data[(data.species_group==SPECIES) & (data.endpoint==PREDICTION_ENDPOINT) & (data.effect==PREDICTION_EFFECT)].Duration_Value.value_counts().index[0]
                except:
                    EXPOSURE_DURATION = default_durations[SPECIES]
                data = combined_results[[SMILES_COLUMN_NAME]]

                results = trident.predict_toxicity(SMILES = data[SMILES_COLUMN_NAME].tolist(), exposure_duration=EXPOSURE_DURATION, endpoint=[PREDICTION_ENDPOINT]*len(data), effect=[PREDICTION_EFFECT]*len(data), return_cls_embeddings=True)
                results.reset_index(drop=True, inplace=True)
                results['exposure_duration'] = EXPOSURE_DURATION
                
                for column in results.columns:
                    results.rename(columns={column: f'{MODEL_VERSION}_{PREDICTION_ENDPOINT}_{PREDICTION_EFFECT} {column}'}, inplace=True)
                
                combined_results[f'{MODEL_VERSION}_{PREDICTION_ENDPOINT}_{PREDICTION_EFFECT} predictions log10(mg/L)'] = results[f'{MODEL_VERSION}_{PREDICTION_ENDPOINT}_{PREDICTION_EFFECT} predictions log10(mg/L)'].tolist()
                combined_results[f'{MODEL_VERSION}_{PREDICTION_ENDPOINT}_{PREDICTION_EFFECT} exposure_duration'] = results[f'{MODEL_VERSION}_{PREDICTION_ENDPOINT}_{PREDICTION_EFFECT} exposure_duration'].tolist()

  0%|          | 0/3 [00:00<?, ?it/s]

EC50EC10_fish EC50 MOR
Renamed NOEC *EC10* in 0 positions



100%|██████████| 26/26 [00:16<00:00,  1.55it/s]


EC50EC10_fish EC50 DVP
Renamed NOEC *EC10* in 0 positions



100%|██████████| 26/26 [00:16<00:00,  1.54it/s]


...


100%|██████████| 26/26 [00:17<00:00,  1.52it/s]


EC50EC10_invertebrates EC50 MOR
Renamed NOEC *EC10* in 0 positions



100%|██████████| 26/26 [00:17<00:00,  1.50it/s]


EC50EC10_invertebrates EC50 DVP
Renamed NOEC *EC10* in 0 positions



 92%|█████████▏| 24/26 [00:16<00:01,  1.52it/s]

In [ ]:
for col in combined_results.columns:
    if 'exposure_duration' in col:
        combined_results[col] = combined_results[col].astype(np.float32)

In [ ]:
combined_results.to_pickle(root+f'data/tutorials/predictions/combined_predictions.pkl.zip', compression='zip')

In [ ]:
combined_results

# Get training data matches

In [ ]:
from inference_utils.pytorch_data_utils import check_training_data, check_training_data_from_scratch

In [ ]:
training_data = pd.read_excel(root+'data/Preprocessed_complete_data.xlsx', sheet_name='dataset')
training_data['species_group'] = training_data.species_group.replace('crustaceans', 'invertebrates')
all_preds = pd.read_pickle(root+f'data/tutorials/predictions/combined_predictions.pkl.zip', compression='zip')

In [ ]:
training_data.species_group.isna()


In [ ]:
training_data.species_group.unique()

In [ ]:
all_preds = check_training_data_from_scratch(all_preds, 'EC50_fish', 'fish', 'EC50', 'MOR', path_to_training_data = root+'data/Preprocessed_complete_data.xlsx')
all_preds

In [ ]:
for model in models:
    for species in tqdm(species_groups):
        MODELTYPE = f'{model}_{species}'
        for endpoint in endpointordering[MODELTYPE]:
            for effect in effectordering[MODELTYPE]:
                all_preds = check_training_data_from_scratch(all_preds, model, species, endpoint, effect, path_to_training_data = root+'data/Preprocessed_complete_data.xlsx')

                for col in all_preds.columns:
                    if ((col == 'species match') | (col == 'endpoint match') | (col == 'effect match')):
                        all_preds.rename(columns={col: f'{model}_{species}_{endpoint}_{effect} {col}'}, inplace=True)

In [ ]:
all_preds

In [ ]:
all_preds.to_pickle(root+f'data/tutorials/predictions/combined_predictions_and_training_data.pkl.zip', compression='zip')

# Add errors from 10x10 CV

In [3]:
import sys
sys.path.insert(1, '/cephyr/users/skall/Alvis/TRIDENT/')
from development.figures.figure_utils.preprocess_data import Preprocess10x10Fold, GroupDataForPerformance
from development.development_utils.preprocessing.Get_data_for_model import PreprocessData
from tqdm.notebook import tqdm

In [4]:
import pandas as pd
training_data = pd.read_pickle(root+'data/tutorials/predictions/combined_predictions_and_training_data.pkl.zip', compression='zip')
training_data['SMILES'] = training_data['SMILES_Canonical_RDKit'].copy()
training_data = PreprocessData(training_data).GetCanonicalSMILES()

In [5]:
def match(x, error_dict):
    try:
        return error_dict[x]
    except:
        return None

In [7]:
for species_group in ['fish', 'invertebrates','algae']:
    for model in tqdm(['EC50','EC10','EC50EC10']):
        if model != 'EC50EC10':
            cvpreds = Preprocess10x10Fold(name=f'{model}_{species_group}', uselogdata=True, full_filepath=root+f'data/results/{model}_{species_group}_predictions_100x_CV_RDkit.pkl.zip')
        else:
            cvpreds = Preprocess10x10Fold(name=f'{model}_{species_group}', uselogdata=True, full_filepath=root+f'data/results/{model}_{species_group}_withoverlap_predictions_100x_CV_RDkit.pkl.zip')
        
        wavgcv = GroupDataForPerformance(cvpreds)
        wavgcv['SMILES'] = wavgcv['Canonical_SMILES_figures'].copy()
        wavgcv = PreprocessData(wavgcv).GetCanonicalSMILES()
        error_dict = dict(zip(wavgcv.SMILES_Canonical_RDKit.tolist(), wavgcv.L1error.tolist()))

        training_data[f'{model}_{species_group} L1Error'] = training_data.SMILES_Canonical_RDKit.apply(lambda x: match(x, error_dict))

  0%|          | 0/3 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: '/cephyr/users/skall/Alvis/Ecotoxformer/inference_scripts_for_website_2/TRIDENT/data/results/EC50_fish_predictions_100x_CV_RDkit.pkl.zip'

In [ ]:
training_data.to_pickle(root+'data/tutorials/combined_predictions_and_errors.pkl.zip', compression='zip')

In [3]:
pd.read_pickle('~/Ecotoxformer/inference_scripts_for_website_2/TRIDENT/data/results/EC50_fish_predictions_100x_CV_RDKit.pkl.zip', compression='zip')

AttributeError: Can't get attribute '_unpickle_block' on <module 'pandas._libs.internals' from '/apps/Arch/software/SciPy-bundle/2020.11-fosscuda-2020b/lib/python3.8/site-packages/pandas/_libs/internals.cpython-38-x86_64-linux-gnu.so'>